In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from pathlib import Path

In [3]:
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split

In [6]:
project_dir = Path(os.path.abspath('')).resolve().parents[0]

#### Read in data

In [7]:
data = pd.read_csv(project_dir / 'data' / 'raw' / 'loan.csv', low_memory=False)

#### Explore data

In [8]:
data.shape

(39717, 111)

In [9]:
data.loan_status.value_counts() / data.shape[0]

Fully Paid     0.829620
Charged Off    0.141677
Current        0.028703
Name: loan_status, dtype: float64

In [10]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


#### Check proportion of missing data

In [11]:
thresh = 0.5

In [12]:
missing_prop = data.isnull().sum() / data.shape[0]

In [13]:
high_prop_miss_cols = missing_prop[missing_prop > thresh].index.values.tolist()

In [14]:
data_sub = data.drop(high_prop_miss_cols, axis=1)

#### Check numeric / non-numeric data

In [15]:
numeric_data = data_sub.select_dtypes(['number'])
numeric_cols = numeric_data.columns.values.tolist()

In [16]:
non_numeric_data = data_sub.drop(numeric_cols, axis=1)

#### Drop irrelevant columns (non-numeric)

In [17]:
drop_non_numeric_cols = ['url', 
                         'desc', 
                         'zip_code', 
                         'addr_state',
                         'earliest_cr_line', 
                         'emp_title', 
                         'verification_status', 
                         'issue_d', 
                         'pymnt_plan', 
                         'initial_list_status',
                         'last_pymnt_d',
                         'last_credit_pull_d',
                         'application_type',
                         'title']

In [18]:
non_numeric_data = non_numeric_data.drop(drop_non_numeric_cols, axis=1)

In [19]:
non_numeric_data.isnull().sum()

term                 0
int_rate             0
grade                0
sub_grade            0
emp_length        1075
home_ownership       0
loan_status          0
purpose              0
revol_util          50
dtype: int64

In [20]:
non_numeric_data = non_numeric_data.fillna('missing')

#### Additional processing on non-numeric data (extract numeric)

In [21]:
additional_numeric_data = non_numeric_data.copy()

In [22]:
additional_numeric_data['term'] = additional_numeric_data['term'].str.extract(pat = '([0-9]+).*')
additional_numeric_data['int_rate'] = additional_numeric_data['int_rate'].str.extract(pat = '([0-9\\.]+).*')
additional_numeric_data['emp_length'] = additional_numeric_data['emp_length'].str.extract(pat = '[^0-9]*([0-9]+).*')
additional_numeric_data['revol_util'] = additional_numeric_data['revol_util'].str.extract(pat = '([0-9\\.]+).*')

In [23]:
non_numeric_data.head()

,term,int_rate,grade,sub_grade,emp_length,home_ownership,loan_status,purpose,revol_util
0,36 months,10.65%,B,B2,10+ years,RENT,Fully Paid,credit_card,83.70%
1,60 months,15.27%,C,C4,< 1 year,RENT,Charged Off,car,9.40%
2,36 months,15.96%,C,C5,10+ years,RENT,Fully Paid,small_business,98.50%
3,36 months,13.49%,C,C1,10+ years,RENT,Fully Paid,other,21%
4,60 months,12.69%,B,B5,1 year,RENT,Current,other,53.90%


In [24]:
additiona_drop_cols = ['home_ownership', 'grade', 'sub_grade', 'loan_status', 'purpose']

In [25]:
additional_numeric_data = additional_numeric_data.drop(additiona_drop_cols, axis=1)

In [26]:
additional_numeric_data = additional_numeric_data.apply(lambda x: x.astype('float'))

In [27]:
additional_numeric_data['emp_length_na'] = additional_numeric_data['emp_length'].isnull() + 0
additional_numeric_data['revol_util_na'] = additional_numeric_data['revol_util'].isnull() + 0

In [28]:
additional_numeric_data = additional_numeric_data.fillna(0)

#### Refine categorical data (one hot encode)

In [29]:
non_numeric_data_final = non_numeric_data.loc[:, additiona_drop_cols]

In [30]:
non_numeric_data_final = non_numeric_data_final.drop('loan_status', axis=1)

In [31]:
non_numeric_data_final_ = pd.get_dummies(non_numeric_data_final)

#### Get response

In [32]:
response = non_numeric_data['loan_status']

#### Drop irrelevant columns (numeric)

In [33]:
drop_numeric_cols = ['id', 
                     'member_id', 
                     'tax_liens', 
                     'delinq_amnt', 
                     'chargeoff_within_12_mths', 
                     'acc_now_delinq',
                     'collections_12_mths_ex_med']

In [34]:
numeric_data = numeric_data.drop(drop_numeric_cols, axis=1)

In [35]:
numeric_data['pub_rec_bankruptcies_na'] = numeric_data['pub_rec_bankruptcies'].isnull() + 0

In [36]:
numeric_data = numeric_data.fillna(0)

#### Stitch data back together

In [37]:
df_final = pd.concat([numeric_data, additional_numeric_data, non_numeric_data_final_, response], axis=1)

In [38]:
df_final.head()

,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,loan_status
0,5000,5000,4975.0,162.87,24000.0,27.65,0,1,3,0,...,0,0,0,0,0,0,0,0,0,Fully Paid
1,2500,2500,2500.0,59.83,30000.0,1.00,0,5,3,0,...,0,0,0,0,0,0,0,0,0,Charged Off
2,2400,2400,2400.0,84.33,12252.0,8.72,0,2,2,0,...,0,0,0,0,0,0,1,0,0,Fully Paid
3,10000,10000,10000.0,339.31,49200.0,20.00,0,1,10,0,...,0,0,0,0,1,0,0,0,0,Fully Paid
4,3000,3000,3000.0,67.79,80000.0,17.94,0,0,15,0,...,0,0,0,0,1,0,0,0,0,Current


In [39]:
df_final.shape

(39717, 93)

In [40]:
(df_final.isnull().sum() != 0).sum()

0

In [41]:
df_final.to_csv(project_dir / 'data' / 'interim' / 'data_clean.csv', index=None)